# Learning a model when the data has noise

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def make_simple_plot():
    fig, axes=plt.subplots(figsize=(12,5), nrows=1, ncols=2);
    axes[0].set_ylabel("$y$")
    axes[0].set_xlabel("$x$")
    axes[1].set_xlabel("$x$")
    axes[1].set_yticklabels([])
    axes[0].set_ylim([-2,2])
    axes[1].set_ylim([-2,2])
    plt.tight_layout();
    return axes
def make_plot():
    fig, axes=plt.subplots(figsize=(20,8), nrows=1, ncols=2);
    axes[0].set_ylabel("$p_R$")
    axes[0].set_xlabel("$x$")
    axes[1].set_xlabel("$x$")
    axes[1].set_yticklabels([])
    axes[0].set_ylim([0,1])
    axes[1].set_ylim([0,1])
    axes[0].set_xlim([0,1])
    axes[1].set_xlim([0,1])
    plt.tight_layout();
    return axes

## Revisiting our model

Let us revisit our model from noiseless_learning.

Let $x$ be the fraction of religious people in a county and $y$ be the probability of voting for Romney as a function of $x$. In other words $y_i$ is data that pollsters have taken which tells us their estimate of people voting for Romney and $x_i$ is the fraction of religious people in county $i$. Because poll samples are finite, there is a margin of error on each data point or county $i$, but we will ignore that for now.

### The setup, again

Let us assume that we have a "population" of 200 counties $x$:

In [3]:
df=pd.read_csv("data/religion.csv")
df.head()

Lets suppose now that the Lord came by and told us that the points in the plot below captures $f(x)$ exactly. 

In [4]:
plt.figure(figsize = (9,6))
x=df.rfrac.values
f=df.promney.values
plt.plot(x,f,'.', alpha=0.3)

Notice that our sampling of $x$ is not quite uniform: there are more points around $x$ of 0.7.

Now, in real life we are only given a sample of points. Lets assume that out of this population of 200 points we are given a sample $\cal{D}$ of 30 data points. Such data is called **in-sample data**. Contrastingly, the entire population of data points is also called **out-of-sample data**.

Here are the indices from this sample.

In [5]:
dfsample = pd.read_csv("data/noisysample.csv")
indexes=dfsample.i.values

In [6]:
samplex = x[indexes]
samplef = f[indexes]

In [7]:
plt.figure(figsize = (9,6))
plt.plot(x,f, 'r.', alpha=0.2, label="population");
plt.plot(samplex,samplef, 's', alpha=0.6, label="in-sample data $\cal{D}$");
plt.legend(loc=4);

## Stochastic Noise and a new $y$

We saw before that $g_{20}$ did a very good job in capturing the curves of the population. However, note that the data obtained from $f$, our target, was still quite smooth. Most real-world data sets are not smooth at all, because of various effects such as measurement errors, other co-variates, and so on. Such **stochastic noise** plays havoc with our fits, as we shall see soon.

Stochastic noise bedevils almost every data set known to humans, and happens for many different reasons. 

Consider for example two customers of a bank with identical credit histories and salaries. One defaults on their mortgage, and the other does not. In this case we have identical $x = (credit, salary)$ for these two customers, but different $y$, which is a variable that is 1 if the customer defaulted and 0 otherwise. The true $y$ here might be a function of other co-variates, such as marital strife, sickness of parents, etc. But, as the bank, we might not have this information. So we get different $y$ for different customers at the information $x$ that we possess.

A similar thing might be happen in the election example, where we have modelled the probability of voting for romney as a function of religiousness of the county. There are many other variables we might not have measured, such as the majority race in that county.  But, we have not measured this information. Thus, in counties with high religiousness fraction $x$ we might have more noise than in others. Consider for example two counties, one with $x=0.8$ fraction of self-identified religious people in the county, and another with $x=0.82$. Based on historical trends, if the first county was mostly white, the fraction of those claiming they would vote for Romney might be larger than in a second, mostly black county. Thus you might have two very $y$'s next to each other on our graphs.

Indeed, we wish to estimate a function $f(x)$ so that the values $y_i$ come from the function $f$. Since we are trying to estimate f with data from only some counties, and furthermore, our estimates of the population behaviour in these counties will be noisy, our estimate wont be the "god given" or "real" f, but rather some **noisy** estimate of it. 


What we have done is introduced a noisy target $y$, so that

$$y = f(x) + \epsilon\,,$$

where $\epsilon$ is a **random** noise term that represents the stochastic noise, or anything that might masquerade as it. 

### Describing things probabilistically

Another way to think about a noisy $y$ is to imagine that our data is generated from a joint probability distribution $P(x,y)$. In our earlier case with no stochastic noise, once you knew $x$, if I were to give you $f(x)$, you could give me $y$ exactly. This is now not possible because of the noise $\epsilon$: we dont know exactly how much noise we have at any given $x$. Thus we need to model $y$ at a given $x$, $P(y \mid x)$, as well using a probability distribution. Since $P(x)$ is also a probability distribution, we have:

$$P(x,y) = P(y \mid x) P(x) .$$

![](images/NoisyModelPxy.png)

Now the entire learning problem can be cast as a problem in probability **density estimation**: if we can estimate $P(x,y)$ and take actions based on that estimate thanks to our risk or error functional, we are done. 

In [8]:
dfpop = pd.read_csv("data/noisypopulation.csv")
dfpop.head()

In [9]:
x = dfpop.x.values
f = dfpop.f.values
y = dfpop.y.values

In [10]:
plt.figure(figsize = (9,6))
plt.plot(x,f, 'r-', alpha=0.6, label="f");
plt.plot(x[indexes], y[indexes], 's', alpha=0.6, label="in-sample y (observed)");
plt.plot(x, y, '.', alpha=0.6, label="population y");
plt.xlabel('$x$');
plt.ylabel('$p_R$')
plt.legend(loc=4);

In the figure above, one can see the scatter of the $y$ population about the curve of $f$, the generating model with no noise. The errors of the 30 observation points ("in-sample") are shown as squares. One can see that observations next to each other can now be fairly different, as we descibed above.

## Fitting a noisy model

Let us now try and fit the noisy data we simulated above, both using straight lines ($\cal{H_1}$), and 20th order polynomials($\cal{H_{20}}$). 


We now fit in both $\cal{H_1}$ and $\cal{H_{20}}$ to find the best fit straight line and best fit 20th order polynomial respectively.

In [11]:
from sklearn.preprocessing import PolynomialFeatures
def polyshape(d, xgrid):
    return PolynomialFeatures(d).fit_transform(xgrid.reshape(-1,1))

In [12]:
from sklearn.linear_model import LinearRegression
def make_predict(inp, target, predx):
    l = LinearRegression(fit_intercept=False)
    return l.fit(inp, target).predict(predx)

In [13]:
g1 = make_predict(x[indexes].reshape(-1,1), f[indexes], x.reshape(-1,1))
g20 = make_predict(polyshape(20, x[indexes]), 
                   f[indexes], polyshape(20, x))

In [14]:
g1noisy = make_predict(x[indexes].reshape(-1,1), y[indexes], x.reshape(-1,1))
g20noisy = make_predict(polyshape(20, x[indexes]), 
                   y[indexes], polyshape(20, x))

In [15]:
plt.figure(figsize = (9,6))
plt.plot(x,f, 'r-', alpha=0.6, label="f");
plt.plot(x[indexes],y[indexes], 's', alpha=0.6, label="y in-sample");
plt.plot(x,g1,  'b--', alpha=0.6, label="$g_1$ (no noise)");
plt.plot(x,g1noisy, 'b:', lw=4, alpha=0.8, label="$g_1$ (noisy)");
plt.legend(loc=4);
plt.ylim([0,1]);
plt.grid()

The noise changes the best fit line ($g_1$) by a little but not by much. The best fit line still does a very poor job of capturing the variation in the data, after the target is noisy.

In [16]:
plt.figure(figsize = (9,6))
plt.plot(x,f, 'r-', alpha=0.6, label="f");
plt.plot(x[indexes],y[indexes], 's', alpha=0.6, label="y in-sample");
plt.plot(x,g20,  'b--', alpha=0.6, label="$g_{20}$ (no noise)");
plt.plot(x,g20noisy, 'b:', lw=4, alpha=0.8, label="$g_{20}$ (noisy)");
plt.legend(loc=4);
plt.ylim([0,1]);
plt.grid()

For the 20th order polynomial, the results are (to put it mildly) very interesting. 

The best fit 20th order polynomial, $g_{20}$, tries to follow all the curves of the observations..in other words, it tries to fit the noise. This is a disaster, as you can see if you plot the population (out-of-sample) points on the plot as well:

In [17]:
plt.figure(figsize = (9,6))
plt.plot(x,f, 'r-', alpha=0.6, label="f");
plt.plot(x[indexes],y[indexes], 's', alpha=0.6, label="y in-sample");
plt.plot(x,y,  '.', alpha=0.6, label="population y");
plt.plot(x,g20noisy, 'b:', alpha=0.6, label="$g_{20}$ (noisy)");
plt.ylim([0,1])
plt.legend(loc=4);

Whoa. The best-fit 20th order polynomial does a reasonable job fitting the in-sample data, and is even well behaved in the middle where we have a lot of in-sample data points. But at places with less in-sample data points, the polynomial wiggles maniacally.

This fitting to the noise is a danger you will encounter again and again in learning. Its called **overfitting**. So, $\cal{H_{20}}$ which seemed to be such a good candidate hypothesis space in the absence of noise, ceases to be one. The take away lesson from this is that we must further ensure that our **model does not fit the noise**.

Lets make a plot similar to the one we made for the deterministic noise earlier, and compare the error in the new $g_1$ and $g_{20}$ fits on the noisy data.

In [18]:
plt.figure(figsize = (9,6))
plt.plot(x, ((g1noisy-f)**2), lw=3, label="$g_1$")
plt.plot(x, ((g20noisy-f)**2), lw=3,label="$g_{20}$");
plt.plot(x, [1]*x.shape[0], "k:", label="noise", alpha=0.2);
for i in indexes[:-1]:
    plt.axvline(x[i], 0, 1, color='r', alpha=0.1)
plt.axvline(x[indexes[-1]], 0, 1, color='r', alpha=0.1, label="samples")
plt.xlabel("$x$")
plt.ylabel("population error")
plt.yscale("log")
plt.legend(loc=4);
plt.title("Noisy Data");

$g_1$ now, for the most part, has a lower error! So you'd be better off by having chosen a set of models with much more bias (the straight lines, $\cal{H}_1$) than a more complex model set ($\cal{H}_{20}$) in the case of noisy data. (except in the high-data center where they are both competitive)

## Overfitting and the Variance of your model

This tendency of a more complex model to overfit, by having enough freedom to fit the noise, is described by something called high **variance**. What is variance?

Variance, simply put, is the "error-bar" or spread in models that would be learnt by training on different data sets $\cal{D_1}, \cal{D_2},...$ drawn from the population. Now, this seems like a circular concept, as in real-life, you do not have access to the population. But since we simulated our data here anyways, we do, and so let us see what happens if we choose **different 30 points randomly from our population of 200**, and fit models in both $\cal{H_1}$ and $\cal{H_{20}}$ to them. We do this on 200 sets of randomly chosen (from the population) data sets of 30 points each and plot the best fit models in noth hypothesis spaces for all 200 sets.

In [19]:
def make_predict_with_model(inp, target, predx):
    l = LinearRegression(fit_intercept=False)
    out = l.fit(inp, target).predict(predx)
    return l, out

In [20]:
def gen(degree, nsims, size, x, out):
    outpoly=[]
    outmodels = []
    for i in range(nsims):
        indexes=np.sort(np.random.choice(x.shape[0], size=size, replace=False))
        l, gnoisy = make_predict_with_model(polyshape(degree, x[indexes]), 
                   y[indexes], polyshape(degree, x))
        outpoly.append(gnoisy)
        outmodels.append(l)
    return outpoly, outmodels

In [21]:
polys1, models1 = gen(1, 200, 30, x, y);
polys20, models20 = gen(20, 200, 30, x, y);

In [22]:
colors = mpl.rcParams['axes.prop_cycle'].by_key()['color']
plt.figure(figsize = (12,8))
axes=make_plot()
axes[0].plot(x,f, 'r-', lw=3, alpha=0.6, label="f");
axes[1].plot(x,f, 'r-', lw=3, alpha=0.6, label="f");
axes[0].plot(x[indexes], y[indexes], 's', alpha=0.6, label="data y");
axes[1].plot(x[indexes], y[indexes], 's', alpha=0.6, label="data y");
axes[0].plot(x, y, '.', alpha=0.6, label="population y");
axes[1].plot(x, y, '.', alpha=0.6, label="population y");
c=colors[2]
for i,p in enumerate(polys1[:-1]):
    axes[0].plot(x,p, alpha=0.05, c=c)
axes[0].plot(x,polys1[-1], alpha=0.05, c=c,label="$g_1$ from different samples")
for i,p in enumerate(polys20[:-1]):
    axes[1].plot(x,p, alpha=0.05, c=c)
axes[1].plot(x,polys20[-1], alpha=0.05, c=c, label="$g_{20}$ from different samples")
axes[0].legend(loc=4);
axes[1].legend(loc=4);

On the left panel, you see the 200 best fit straight lines, each a fit on a different 30 point training sets from the 200 point population. The best-fit lines bunch together, even if they dont quite capture $f$ (the thick red line) or the data (squares) terribly well.

On the right panel, we see the same with best-fit models chosen from $\cal{H}_{20}$. It is a diaster. While most of the models still band around the central trend of the real curve $f$ and data $y$ (and you still see the waves corresponding to all too wiggly 20th order polynomials), a substantial amount of models veer off into all kinds of noisy hair all over the plot. This is **variance**: the the predictions at any given $x$ are all over the place.

The variance can be seen in a different way by plotting the coefficients of the polynomial fit. Below we plot the coefficients of the fit in $\cal{H}_1$. The variance is barely 0.2 about the mean for both co-efficients.

In [23]:
parray1=np.empty((200, 2))
parray20=np.empty((200, 21))
for j, p in enumerate(models1):
    m1 = models1[j]
    m20 = models20[j]
    parray1[j, :] = m1.coef_
    parray20[j, :] = m20.coef_
    

In [24]:
dict1 = {"coef {}".format(i):parray1[:,i] for i in range(2)}
df1 = pd.DataFrame(dict1)
df1

In [25]:
dict20 = {"coef {}".format(i):parray20[:,i] for i in range(21)}
df20 = pd.DataFrame(dict20)
df20

In [28]:
import seaborn as sns
fig = plt.figure(figsize=(14, 5)) 
from matplotlib import gridspec
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 10]) 
axes = [plt.subplot(gs[0]), plt.subplot(gs[1])]
axes[0].set_ylabel("value")
axes[0].set_xlabel("coeffs")
axes[1].set_xlabel("coeffs")
plt.tight_layout();
sns.violinplot(data=df1, ax=axes[0]);
sns.violinplot(data=df20, ax=axes[1]);
axes[0].set_yscale("symlog");
axes[1].set_yscale("symlog");
axes[0].set_ylim([-1e13, 1e13]);
axes[1].set_ylim([-1e13, 1e13]);

In the right panel we plot the coefficients of the fit in $\cal{H}_{20}$. This is why we use the word "variance": the spread in the values of the middle coefficients about their means (dashed lines) is of the order $10^{10}$ (the vertical height of the bulges), with huge outliers!! The 20th order polynomial fits are a disaster!

## The process of learning, again

The process of learning has two parts:

1. Fit for a model by minimizing the in-sample risk
2. Hope that the in-sample risk approximates the out-of-sample risk well.

Mathematically, we are saying that:

$$
\begin{eqnarray*}
A &:& R_{\cal{D}}(g) \,\,smallest\,on\,\cal{H}\\
B &:& R_{out} (g) \approx R_{\cal{D}}(g)
\end{eqnarray*}
$$


Well, we are scientists. Just hoping does not befit us. But we only have a sample. What are we to do? We can model the in-sample risk and out-of-sample risk. And we can use a test set to estimate our out of sample risk, as we see [here](testingtraining.html).